<a href="https://colab.research.google.com/github/cramosme/LLM-Phishing-Detection/blob/main/LLM_Phishing_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Clone git repo, comment out after running once
# !git clone https://github.com/cramosme/LLM-Phishing-Detection.git

# %cd LLM-Phishing-Detection

#!ls

In [50]:
import pandas as pd

# Load the CSVs
dfLLMPhish = pd.read_csv(f"/content/LLM-Phishing-Detection/data/llm-generated/phishing-fixed.csv") # Have to use Fixed version and not original because original didnt quote text correctly so parsing was messed up
print(f"Loaded {len(dfLLMPhish)} LLM Generated phishing emails")
display(dfLLMPhish.info())

print("\n")

dfLLMLegit = pd.read_csv(f"/content/LLM-Phishing-Detection/data/llm-generated/legit-fixed.csv") # Have to use Fixed version and not original because original didnt quote text correctly so parsing was messed up
print(f"Loaded {len(dfLLMLegit)} LLM Generated legit emails")
display(dfLLMLegit.info())

print("\n")

dfHumanPhish = pd.read_csv(f"/content/LLM-Phishing-Detection/data/human-generated/phishing.csv") # Issue only occured with the LLM ones, human generated we can just use the original dataset's files
print(f"Loaded {len(dfHumanPhish)} Human Generated phishing emails")
display(dfHumanPhish.info())

print("\n")

dfHumanLegit = pd.read_csv(f"/content/LLM-Phishing-Detection/data/human-generated/legit.csv")
print(f"Loaded {len(dfHumanLegit)} Human Generated legit emails")
display(dfHumanLegit.info())

Loaded 1000 LLM Generated phishing emails
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


None



Loaded 1000 LLM Generated legit emails
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


None



Loaded 1000 Human Generated phishing emails
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    1000 non-null   object
 1   receiver  978 non-null    object
 2   date      1000 non-null   object
 3   subject   998 non-null    object
 4   body      1000 non-null   object
 5   urls      1000 non-null   int64 
 6   label     1000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


None



Loaded 1000 Human Generated legit emails
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    1000 non-null   object
 1   receiver  984 non-null    object
 2   date      1000 non-null   object
 3   subject   1000 non-null   object
 4   body      1000 non-null   object
 5   urls      1000 non-null   object
 6   label     1000 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


None

## Printing out the value count to check that all 1000 values are the same. Phish is 1. Legit is 0.

In [51]:
dfLLMPhish['label'].value_counts()

,count
label,
1,1000


In [52]:
dfLLMLegit['label'].value_counts()

,count
label,
0,1000


In [53]:
dfHumanPhish['label'].value_counts()

,count
label,
1,1000


In [54]:
dfHumanLegit['label'].value_counts()

,count
label,
0,1000
